Input and output file paths

In [1]:
input_file="C:/Users/yrui7/Downloads/BillGates_2010.wav"
output_file_path = "C:/Users/yrui7/Downloads/recognized_conversation_text.txt"  

Speech to text

In [2]:
import azure.cognitiveservices.speech as speechsdk
import os
import time
from dotenv import load_dotenv
load_dotenv()

output_file_path = output_file_path
transcription_output = ""


def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    print('Canceled event')

def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    global transcription_output
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        #print('\tSpeaker ID={}:'.format(evt.result.speaker_id))
        #print('\t{}'.format(evt.result.text))
        transcription_output += f"Speaker ID={evt.result.speaker_id}:\n{evt.result.text}\n\n"

    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        #print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))
        transcription_output += f"NOMATCH: Speech could not be TRANSCRIBED: {evt.result.no_match_details}\n\n"

def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')
    
def recognize_from_file(input_file):
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.getenv('SPEECH_SERVICE_KEY'), region=os.getenv('SPEECH_REGION'))
    speech_config.speech_recognition_language="en-US"

    audio_config = speechsdk.audio.AudioConfig(filename=input_file)
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(speech_config=speech_config, audio_config=audio_config)

    transcribing_stop = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        #"""callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal transcribing_stop
        transcribing_stop = True

    # Connect callbacks to the events fired by the conversation transcriber
    conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
    conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
    conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
    conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)
    # stop transcribing on either session stopped or canceled events
    conversation_transcriber.session_stopped.connect(stop_cb)
    conversation_transcriber.canceled.connect(stop_cb)

    conversation_transcriber.start_transcribing_async()

    # Waits for completion.
    while not transcribing_stop:
        time.sleep(.5)

    conversation_transcriber.stop_transcribing_async()

recognize_from_file(input_file)

with open(output_file_path, 'w') as output_file:
    output_file.write(transcription_output)

print("Saved content:\n", transcription_output)


SessionStarted event


KeyboardInterrupt: 

Text Summarization

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential



def sample_abstractive_summarization() -> None:
    endpoint = os.getenv('AZURE_LANGUAGE_ENDPOINT')
    key = os.getenv('AZURE_LANGUAGE_KEY')

    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )
    # Replace this with the recognized text you want to summarize
    #recognized_text = open_file(output_file)

    document = [ {"id": "1", "text": transcription_output}]

    poller = text_analytics_client.begin_abstract_summary(document)
    abstract_summary_results = poller.result()
    for result in abstract_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            [print(f"{summary.text}\n") for summary in result.summaries]
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))
    # [END abstract_summary]
    
sample_abstractive_summarization()



Summaries abstracted:
The speaker discusses the importance of energy and climate change, particularly for the world's poorest 2 billion people. They highlight the need to reduce the amount of CO2 emitted from energy production to zero, which will significantly impact the planet's natural ecosystems. The speaker suggests five potential energy solutions: fossil fuels, renewable sources, geothermal, fusion, and biofuels, each with their own challenges. They also mention the potential of nuclear energy, particularly the concept of Tarrant power, which could provide significant energy storage and waste disposal.

